In [1]:
import scanpy as sc
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier


In [2]:
human_ad = sc.read("export_data/uce_tabula_33_8_umap.h5ad")
fly_ad = sc.read("export_data/fly_cell_atlas_uce_adata_33layer_umap.h5ad")

/lfs/ampere5/0/yanay/env/micromamba/envs/dogma/lib/python3.11/site-packages/anndata/_core/anndata.py:1825: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [3]:
fly_gut_ad = fly_ad[fly_ad.obs["tissue"] == "gut"].copy()
vcs = fly_gut_ad.obs["consensus_community_annotation"].value_counts()
keep_cts = list(vcs[vcs >= 100].index)
fly_gut_ad = fly_gut_ad[fly_gut_ad.obs["consensus_community_annotation"].isin(keep_cts)]

fly_gut_ad.obs["consensus_community_annotation"].value_counts()

consensus_community_annotation
crop                                              4315
Enterocyte-like                                   1150
cardia, R0 midgut region                           978
enterocyte of anterior adult midgut epithelium     734
intestinal stem cell                               299
hindgut                                            278
adult differentiating enterocyte                   272
visceral muscle                                    253
middle midgut                                      229
enteroblast                                        224
visceral muscle of the crop                        200
adult hindgut epithelium                           184
Name: count, dtype: int64

# Compare Cell types to gut

In [4]:
neigh = KNeighborsClassifier(n_neighbors=1)

In [5]:
train_x = human_ad.X # uce embeddings
train_y = human_ad.obs["cell_ontology_class"]

In [6]:
neigh.fit(train_x, train_y)

,"n_neighbors n_neighbors: int, default=5Number of neighbors to use by default for :meth:`kneighbors` queries.",1
,"weights weights: {'uniform', 'distance'}, callable or None, default='uniform'Weight function used in prediction. Possible values:- 'uniform' : uniform weights. All points in each neighborhood are weighted equally.- 'distance' : weight points by the inverse of their distance. in this case, closer neighbors of a query point will have a greater influence than neighbors which are further away.- [callable] : a user-defined function which accepts an array of distances, and returns an array of the same shape containing the weights.Refer to the example entitled:ref:`sphx_glr_auto_examples_neighbors_plot_classification.py`showing the impact of the `weights` parameter on the decisionboundary.",'uniform'
,"algorithm algorithm: {'auto', 'ball_tree', 'kd_tree', 'brute'}, default='auto'Algorithm used to compute the nearest neighbors:- 'ball_tree' will use :class:`BallTree`- 'kd_tree' will use :class:`KDTree`- 'brute' will use a brute-force search.- 'auto' will attempt to decide the most appropriate algorithm based on the values passed to :meth:`fit` method.Note: fitting on sparse input will override the setting ofthis parameter, using brute force.",'auto'
,"leaf_size leaf_size: int, default=30Leaf size passed to BallTree or KDTree. This can affect thespeed of the construction and query, as well as the memoryrequired to store the tree. The optimal value depends on thenature of the problem.",30
,"p p: float, default=2Power parameter for the Minkowski metric. When p = 1, this is equivalentto using manhattan_distance (l1), and euclidean_distance (l2) for p = 2.For arbitrary p, minkowski_distance (l_p) is used. This parameter is expectedto be positive.",2
,"metric metric: str or callable, default='minkowski'Metric to use for distance computation. Default is ""minkowski"", whichresults in the standard Euclidean distance when p = 2. See thedocumentation of `scipy.spatial.distance`_ andthe metrics listed in:class:`~sklearn.metrics.pairwise.distance_metrics` for valid metricvalues.If metric is ""precomputed"", X is assumed to be a distance matrix andmust be square during fit. X may be a :term:`sparse graph`, in whichcase only ""nonzero"" elements may be considered neighbors.If metric is a callable function, it takes two arrays representing 1Dvectors as inputs and must return one value indicating the distancebetween those vectors. This works for Scipy's metrics, but is lessefficient than passing the metric name as a string.",'minkowski'
,"metric_params metric_params: dict, default=NoneAdditional keyword arguments for the metric function.",None
,"n_jobs n_jobs: int, default=NoneThe number of parallel jobs to run for neighbors search.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.Doesn't affect :meth:`fit` method.",None


In [7]:
fly_gut_predicted = pd.Series(neigh.predict(fly_gut_ad.obsm["X_uce"]))
fly_gut_ad.obs["predicted_cell_type"] = fly_gut_predicted.values
fly_gut_predicted.value_counts()

/tmp/user/21290/ipykernel_929134/2679500750.py:2: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  fly_gut_ad.obs["predicted_cell_type"] = fly_gut_predicted.values
/lfs/ampere5/0/yanay/env/micromamba/envs/dogma/lib/python3.11/site-packages/anndata/_core/anndata.py:1825: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/lfs/ampere5/0/yanay/env/micromamba/envs/dogma/lib/python3.11/site-packages/anndata/_core/anndata.py:1825: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


macrophage                                            1204
smooth muscle cell                                    1179
enterocyte of epithelium proper of small intestine    1043
fat cell                                               951
bladder urothelial cell                                944
cardiac endothelial cell                               812
stromal cell                                           811
male germ cell                                         440
epithelial cell                                        375
supporting cell of vestibular epithelium               271
regular atrial cardiac myocyte                         251
skeletal muscle satellite stem cell                    135
plasma cell                                             98
acinar cell                                             88
ventricular cardiac muscle cell                         81
tracheal goblet cell                                    54
t cell                                                  

In [8]:
ct_map = {"Enterocyte-like":"Enterocyte",
"enterocyte of anterior adult midgut epithelium":"Enterocyte",
"crop":"crop",
"adult differentiating enterocyte":"Enterocyte",
"enteroblast":"Enterocyte",
"middle midgut":"Midgut",
"cardia, R0 midgut region":"Cardia",
"intestinal stem cell":"ISC",
"visceral muscle of the crop":"visceral muscle",
"hindgut":"hindgut",
"visceral muscle":"visceral muscle",
"adult hindgut epithelium":"hindgut"}

In [9]:
fly_gut_ad.obs["mapped_ct"] = [ct_map.get(a) for a in fly_gut_ad.obs["consensus_community_annotation"]]

In [10]:
fly_gut_ad[fly_gut_ad.obs["mapped_ct"] == "Enterocyte"].obs[["predicted_cell_type"]].value_counts()

predicted_cell_type                               
enterocyte of epithelium proper of small intestine    501
fat cell                                              341
epithelial cell                                       256
macrophage                                            222
smooth muscle cell                                    174
stromal cell                                          171
cardiac endothelial cell                              167
supporting cell of vestibular epithelium              137
male germ cell                                        132
bladder urothelial cell                                58
skeletal muscle satellite stem cell                    44
acinar cell                                            22
regular atrial cardiac myocyte                         21
enteroendocrine cell of small intestine                20
tracheal goblet cell                                   18
t cell                                                 14
plasma cell          

In [11]:
len(human_ad.obs["tissue"].unique())

28

In [12]:
len(human_ad.obs["cell_ontology_class"].unique())

180